# python装饰器的应用

## 情景描述

有一个函数，存在一定概率会运行失败。

In [3]:
def prob_fail():
    from random import randint
    if randint(0,100) < 20:
        print('Succeed')
    else:
        print('Fail')
        raise Exception

In [4]:
prob_fail()

Fail


Exception: 

In [9]:
prob_fail()

Succeed


## 希望能达到的效果是，在函数运行失败时自动重试。
### 不用装饰器也可实现

In [10]:
while True:
    try:
        prob_fail()
        break
    except Exception as e:
        print('Retrying. ')

Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Succeed


### 若多个函数都想实现此功能，用装饰器更方便
装饰器的特征：
* Closure 闭包函数
* 外层函数的参数只有一个，且是需要被装饰的函数

In [11]:
def decorator(func):
    def inner():
        while True:
            try:
                return func()
            except Exception as e:
                print('Retrying. ')
    return inner

@decorator
def prob_fail():
    from random import randint
    if randint(0,100) < 20:
        print('Succeed')
    else:
        print('Fail')
        raise Exception

__之后只要直接调用函数本身即可。__

In [13]:
prob_fail()

Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Fail
Retrying. 
Succeed


### 若函数本身有参数，则需要在inner函数中补齐参数设置，以满足不同函数的调用需求

In [14]:
def decorator(func):
    def inner(*args, **kargs):
        while True:
            try:
                return func(*args, **kargs)
            except Exception as e:
                print('Retrying. ')
    return inner

### 若想要在装饰器中增加别的参数以实现更多控制，则需要再包一层函数

此处设置了__最大重试次数(max_retry_n)__和__重试等待时间(retry_wait_arg)__的控制。  
若超过__最大重试次数__，则抛出MaxRetryNumError，这是Exception类的一个子类。

In [15]:
class MaxRetryNumError(Exception):
    pass

def retry_control(max_retry_n, retry_wait_arg):
    from time import sleep
    def decorator(func):
        def inner(*args, **kargs):
            i = 0
            while True:
                try:
                    return func(*args, **kargs)

                except Exception as e:
                    wait_sec = retry_wait_arg*(i**2)
                    print('Wait {} seconds before retrying. '.format(wait_sec))
                    sleep(wait_sec)

                i += 1
                if i > max_retry_n:
                    raise MaxRetryNumError('Reach the Max Retry Number: {}! '.format(max_retry_n))
                print('Retry Time {}'.format(i))

        return inner
    return decorator

In [16]:
@retry_control(3, 3)
def prob_fail():
    from random import randint
    if randint(0,100) < 20:
        print('Succeed')
    else:
        print('Fail')
        raise Exception

In [18]:
prob_fail()

Fail
Wait 0 seconds before retrying. 
Retry Time 1
Fail
Wait 3 seconds before retrying. 
Retry Time 2
Fail
Wait 12 seconds before retrying. 
Retry Time 3
Fail
Wait 27 seconds before retrying. 


MaxRetryNumError: Reach the Max Retry Number: 3! 